[View in Colaboratory](https://colab.research.google.com/github/hsarena/dl-rnn/blob/master/simple_gan_mnist.ipynb)

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


DC_D_W1 = tf.Variable(xavier_init([5, 5, 1, 16]))
DC_D_b1 = tf.Variable(tf.zeros(shape=[16]))

DC_D_W2 = tf.Variable(xavier_init([3, 3, 16, 32]))
DC_D_b2 = tf.Variable(tf.zeros(shape=[32]))

DC_D_W3 = tf.Variable(xavier_init([7 * 7 * 32, 128]))
DC_D_b3 = tf.Variable(tf.zeros(shape=[128]))

DC_D_W4 = tf.Variable(xavier_init([128, 1]))
DC_D_b4 = tf.Variable(tf.zeros(shape=[1]))

theta_DC_D = [DC_D_W1, DC_D_b1, DC_D_W2, DC_D_b2, DC_D_W3, DC_D_b3, DC_D_W4, DC_D_b4]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit


def dc_generator(z):
    pass


def dc_discriminator(x):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    conv1 = tf.nn.relu(tf.nn.conv2d(x, DC_D_W1, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b1)
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, DC_D_W2, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b2)
    conv2 = tf.reshape(conv2, shape=[-1, 7 * 7 * 32])
    h = tf.nn.relu(tf.matmul(conv2, DC_D_W3) + DC_D_b3)
    logit = tf.matmul(h, DC_D_W4) + DC_D_b4
    prob = tf.nn.sigmoid(logit)

    return prob, logit


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


G_sample = generator(Z)
D_real, D_logit_real = dc_discriminator(X)
D_fake, D_logit_fake = dc_discriminator(G_sample)

#D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
#G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_DC_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

#mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

if not os.path.exists('../out/'):
    os.makedirs('../out/')

i = 0

for it in range(1 000):
    if it % 100 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('../out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0
D loss: 3.167
G_loss: 37.97

Iter: 100
D loss: 7.402e-05
G_loss: 27.66

Iter: 200
D loss: 0.001545
G_loss: 11.66

Iter: 300
D loss: 0.01108
G_loss: 19.16

Iter: 400
D loss: 2.551e-09
G_loss: 43.55

Iter: 500
D loss: 6.051e-07
G_loss: 19.99

Iter: 600
D loss: 3.659e-06
G_loss: 25.08

Iter: 700
D loss: 0.03037
G_loss: 30.74

Iter: 800
D loss: 0.02667
G_loss: 26.43

Iter: 900
D loss: 0.005999
G_loss: 33.77

Iter: 1000
D loss: 3.895e-09
G_loss: 31.07

Iter: 1100
D loss: 0.1251
G_loss: 83.05

Iter: 1200
D loss: 0.001065
G_loss: 49.12

Iter: 1300
D loss: 0.001385
G_loss: 21.4

Iter: 1400
D loss: 2.56e-05
G_loss: 16.07

Iter: 1500
D loss: 0.1012
G_loss: 37.73

Iter: 1600
D loss: 4.963e-15
G_loss: 47.13

Iter: 1700
D loss: 0.6673
G_loss: 85.36

Iter: 1800
D loss: 0.0009513
G_loss: 38.37

Iter: 1

Iter: 6400
D loss: 0.04897
G_loss: 9.006

Iter: 6500
D loss: 0.1027
G_loss: 8.626

Iter: 6600
D loss: 0.1022
G_loss: 7.778

Iter: 6700
D loss: 0.1268
G_loss: 13.0

Iter: 6800
D loss: 0.1032
G_loss: 7.13

Iter: 6900
D loss: 0.05467
G_loss: 12.71

Iter: 7000
D loss: 0.04983
G_loss: 6.844

Iter: 7100
D loss: 0.1963
G_loss: 7.574

Iter: 7200
D loss: 0.03357
G_loss: 6.813

Iter: 7300
D loss: 0.07454
G_loss: 8.56

Iter: 7400
D loss: 0.1017
G_loss: 6.817

Iter: 7500
D loss: 0.1035
G_loss: 4.674

Iter: 7600
D loss: 0.02348
G_loss: 11.16

Iter: 7700
D loss: 0.1962
G_loss: 5.99

Iter: 7800
D loss: 0.1607
G_loss: 8.78

Iter: 7900
D loss: 0.09824
G_loss: 5.441

Iter: 8000
D loss: 0.04521
G_loss: 8.385

Iter: 8100
D loss: 0.2691
G_loss: 6.247

Iter: 8200
D loss: 0.1237
G_loss: 7.926

Iter: 8300
D loss: 0.02859
G_loss: 7.071

Iter: 8400
D loss: 0.08629
G_loss: 7.878

Iter: 8500
D loss: 0.1026
G_loss: 6.64

Iter: 8600
D loss: 0.1273
G_loss: 7.254

Iter: 8700
D loss: 0.05113
G_loss: 6.68

Iter: 8800
D

Iter: 13300
D loss: 0.2147
G_loss: 4.389

Iter: 13400
D loss: 0.2287
G_loss: 5.574

Iter: 13500
D loss: 0.1901
G_loss: 4.519

Iter: 13600
D loss: 0.2813
G_loss: 3.858

Iter: 13700
D loss: 0.1679
G_loss: 4.937

Iter: 13800
D loss: 0.1495
G_loss: 5.246

Iter: 13900
D loss: 0.1695
G_loss: 4.148

Iter: 14000
D loss: 0.1193
G_loss: 4.579

Iter: 14100
D loss: 0.1327
G_loss: 3.846

Iter: 14200
D loss: 0.09371
G_loss: 4.448

Iter: 14300
D loss: 0.1466
G_loss: 4.747

Iter: 14400
D loss: 0.1637
G_loss: 4.295

Iter: 14500
D loss: 0.2306
G_loss: 4.323

Iter: 14600
D loss: 0.1441
G_loss: 4.157

Iter: 14700
D loss: 0.1759
G_loss: 4.115

Iter: 14800
D loss: 0.1169
G_loss: 4.288

Iter: 14900
D loss: 0.1529
G_loss: 3.891

